![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [3]:

from AlgorithmImports import *
# Quick end-to-end test for V-bar footprint aggregation and storage
from datetime import date
import pandas as pd

from orchestrator import run
from footprint_storage import get_year_file_path

# Assumes `qb` is available in the Research environment
qb = QuantBook()


symbol = qb.add_future(Futures.Indices.NASDAQ_100_E_MINI, Resolution.SECOND).symbol
# Futures.Indices.NASDAQ_100_E_MINI

start = date(2016, 1, 1)
end = date(2016, 12, 31)

v_unit = 1000  # minimal volume unit per bar
# Set tick_size per instrument; examples: NQ=0.25, GC=0.1

sec = qb.Securities[symbol]
tick_size = sec.SymbolProperties.MinimumPriceVariation


run(qb=qb, symbol=symbol, start_date=start, end_date=end, v_unit=v_unit, tick_size=tick_size, force_recompute=False)

print("Done.")


Done.


In [4]:
# 从我们刚刚创建的文件中导入校验函数
from validator import validate_daily_open


# --- 执行 ---
print(f"开始校验合约: {symbol}")
print(f"日期范围: {start} to {end}")

# ============== Cell 2: 运行校验 ==============
print("\n正在运行校验...")
# data_root 参数可以按需修改，这里使用模块中的默认值 '/LeanCLI/footprint_data'
validation_errors = validate_daily_open(
    qb=qb,
    symbol=symbol,
    start_date=start,
    end_date=end
)
print("校验完成。")


# ============== Cell 3: 显示结果 ==============
print("\n--- 校验结果 ---")
if not validation_errors:
    print("✅ 校验通过！在指定日期范围内，所有日期的开盘价均在2个tick的容忍误差内。")
else:
    print(f"❌ 校验发现 {len(validation_errors)} 个问题。")
    
    # 将结果转换为 DataFrame 以便清晰展示
    errors_df = pd.DataFrame(validation_errors)
    
    # 计算差异的tick数量，以便更直观地判断
    if "difference" in errors_df.columns and "tick_size" in errors_df.columns:
        # 使用 .loc 避免 SettingWithCopyWarning
        errors_df.loc[:, "difference_in_ticks"] = errors_df["difference"] / errors_df["tick_size"]
    
    # 为了更好的可读性，重新排列一下列的顺序
    cols_order = [
        "date", "status", "daily_open", "daily_open_time", "footprint_open", "footprint_open_time", 
        "difference", "tick_size", "difference_in_ticks", "message"
    ]
    
    # 过滤掉在DataFrame中不存在的列
    existing_cols = [col for col in cols_order if col in errors_df.columns]
    
    print("\n详细信息:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        df_to_print = errors_df[existing_cols].copy()

        # 检查并格式化时间列
        if 'daily_open_time' in df_to_print.columns:
            df_to_print['daily_open_time'] = pd.to_datetime(df_to_print['daily_open_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
        if 'footprint_open_time' in df_to_print.columns:
            df_to_print['footprint_open_time'] = pd.to_datetime(df_to_print['footprint_open_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

        print(df_to_print.to_string())

开始校验合约: /NQ
日期范围: 2016-01-01 to 2016-12-31

正在运行校验...
获取了 354375 个分钟 bar，正在准备校验...
获取了 55910 个 footprint bars，分布在 309 个交易日中。
共校验了 309 个交易日。
校验完成。

--- 校验结果 ---
❌ 校验发现 1 个问题。

详细信息:
         date    status  daily_open      daily_open_time  footprint_open  footprint_open_time  difference  tick_size  difference_in_ticks
0  2016-01-21  Mismatch      4135.0  2016-01-21 00:00:00         4135.75  2016-01-21 00:00:01        0.75       0.25                  3.0
